# Graficacion Meteorologica Mediante Interpolaciones 🌡

<img src=Imagenes/App2.jpg width=300px>
<h3>Intregrantes:</h3>
<h4>Juan Vargas <br>
Freddy Jaimes <br>
Angel de Leon</h4>
<hr>

<h2>Tomamos los datos consumiendo una api del clima</h2>
<h5>https://developer.weatherunlocked.com/documentation/localweather/forecast</h5>
<h3>Elegimos la locacion de Turquia estambul</h3>
<img src=Imagenes/MemeTurco.jpg width=300px>
<hr>

## Consumo De Api para la Generacion de datos (1 Semana)

In [37]:
#Importamos las librerias necesarias
using HTTP
using JSON

#Pido los datos desde la api
url = "http://api.weatherunlocked.com/api/forecast/41.01384,-28.94966?app_id=f8341b8f&app_key=380d876792d6ac1b32c5258c07c010a4"
res = HTTP.get(url)
json = JSON.parse(String(res.body))

Temperatura_max = []
Temperatura_min = []
temperatura = []
x = json["Days"]

for i in x
    push!(Temperatura_max, i["temp_max_c"])
    push!(Temperatura_min, i["temp_min_c"])
end

for i in 1:1:length(Temperatura_max)
    temp_max = Temperatura_max[i]
    temp_min = Temperatura_min[i]
    prom = temp_max + temp_min
    push!(temperatura, round(prom/2 , digits=2))
end
print(temperatura)
typeof(temperatura)

Any[20.65, 22.5, 22.35, 22.45, 23.7, 22.6, 20.2]

Vector{Any} (alias for Array{Any, 1})

## Datos Meteorologicos anuales de Estambul

In [ ]:
api = "790e4819ffc841c5b1810355241106"
loc = "Istanbul"
doc = loc * "_weather_data.json"

data = []

for m in 1:12
    s_date = "2023-$(lpad(m, 2, '0'))-01"
    e_date = m != 12 ? "2023-$(lpad(m, 2, '0'))-31" : "2023-12-31"

    url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=$(loc)&tp=24&date=$(s_date)&enddate=$(e_date)&format=json&key=$(api)"

    res = HTTP.get(url)

    if res.status == 200
        d = JSON.parse(String(res.body))
        push!(data, d)
        println("Datos del mes $m obtenidos correctamente")
    else
        println("Error al obtener datos del mes $m: $(res.status)")
    end
end

meses = [
    "Enero",
    "Febrero",
    "Marzo",
    "Abril",
    "Mayo",
    "Junio",
    "Julio",
    "Agosto",
    "Septiembre",
    "Octubre",
    "Noviembre",
    "Diciembre",
]

max_t = []
min_t = []
avg_t = []

for m_data in data
    m_max_t = []
    m_min_t = []
    m_avg_t = []

    for day in m_data["data"]["weather"]
        max_t = day["maxtempC"]
        min_t = day["mintempC"]
        avg_t = day["avgtempC"]

        push!(m_max_t, max_t)
        push!(m_min_t, min_t)
        push!(m_avg_t, avg_t)
    end

    push!(max_t, m_max_t)
    push!(min_t, m_min_t)
    push!(avg_t, m_avg_t)
end



<hr>

## Interpolacion de Taylor

In [35]:
using SymPy
using Plots

function Taylor(temperatura, tiempo, x0)

    x = symbols('x')
    c_temperatura = [temperatura]
    
    for i in 1:(length(temperatura)-1)
        push!(c_temperatura, diff(c_temperatura[i]))
    end
    taylor = c_temperatura[1][1]
    fact = 1
    f_pol = 0
    for i in 1:(length(temperatura)-1)
        fact *= 1
        pol = c_temperatura[i][i+1] / fact
        for j in 1:(i-1)
            pol *= (x - tiempo[j+1])
        end
        f_pol = f_pol + pol
    end
    result = subs(f_pol, x, x0)
    return f_pol, result, x0
end
x = 1:length(temperatura)

taylor , result , x0 = Taylor(temperatura, x, 3)
# Graficamos la funcion de temperaturas por los dias y el polinomio de taylor
plot(x,temperatura, label="Datos originales", lw=3, marker=:circle)
plot!(taylor, label="Interpolación de Taylor", lw=3, color=:green)
xlabel!("Días")
ylabel!("Temperatura (°C)")
title!("Interpolación de Taylor")
ylims!(0, 40)
xlims!(1, 8)


LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\ElAdagioDeJP\.julia\packages\PyCall\1gn3u\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'RecursionError'>
RecursionError('maximum recursion depth exceeded while calling a Python object')
  File "C:\Users\ElAdagioDeJP\.julia\conda\3\x86_64\lib\site-packages\sympy\core\symbol.py", line 814, in symbols
    result.append(symbols(name, cls=cls, **args))
  File "C:\Users\ElAdagioDeJP\.julia\conda\3\x86_64\lib\site-packages\sympy\core\symbol.py", line 814, in symbols
    result.append(symbols(name, cls=cls, **args))
  File "C:\Users\ElAdagioDeJP\.julia\conda\3\x86_64\lib\site-packages\sympy\core\symbol.py", line 814, in symbols
    result.append(symbols(name, cls=cls, **args))
  [Previous line repeated 996 more times]
  File "C:\Users\ElAdagioDeJP\.julia\conda\3\x86_64\lib\site-packages\sympy\core\symbol.py", line 724, in symbols
    if isinstance(names, str):
